In [40]:
from plyfile import PlyData, PlyElement
import numpy as np

In [41]:
plydata = PlyData.read('dragon_vrip_res2.ply')

In [42]:
len(plydata)

2

In [43]:
print(plydata)

ply
format ascii 1.0
comment generated by ply_writer
element vertex 100250
property float x
property float y
property float z
element face 202520
property list uchar int vertex_indices
end_header


In [44]:
plydata['vertex']

PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float')), count=100250, comments=[])

In [45]:
plydata['vertex'][1]

(0.0330387, 0.056612, -0.0501226)

In [46]:
plydata['face'][0]

(array([1, 2, 3], dtype=int32),)

In [47]:
count = 0
c_x = 0;
c_y = 0;
c_z = 0;
for x in plydata['vertex']:
#     print(x[0], x[1], x[2])
#     print(str([x[0], x[1], x[2]]))
    c_x += x[0]
    c_y += x[1]
    c_z += x[2]
    count += 1

In [48]:
count

100250

In [49]:
c_x = c_x / count;
c_y = c_y / count;
c_z = c_z / count;

In [50]:
mesh_center = np.array([c_x, c_y, c_z])

In [51]:
print(mesh_center)

[-0.01087445  0.10929135 -0.0082076 ]


In [52]:
#calculate and store area-weighted vertex normals for future use#
vertex_normals = np.zeros((count, 3))
for x in plydata['face']:
    #get indexs of vertices in the face #
    index = x[0]
    
    #get corresponding vertices# 
    point_a = plydata['vertex'][index[0]]
    point_b = plydata['vertex'][index[1]]
    point_c = plydata['vertex'][index[2]]
    
    #use np to reconstruct to eliminate extra properties rather than xyz and vector calculation#
    t_point_a = np.array([point_a[0], point_a[1], point_a[2]])
    t_point_b = np.array([point_b[0], point_b[1], point_b[2]])
    t_point_c = np.array([point_c[0], point_c[1], point_c[2]])
    
    #construct vectors in the face#
    v1 = t_point_b - t_point_a
    v2 = t_point_c - t_point_a
    
    #calculate normal and then normalize#
    normal = np.cross(v1, v2)
    normal = normal / (np.linalg.norm(normal))


    v_t = t_point_a - mesh_center
#     if (np.dot(v_t, normal) < 0):
#         normal =  np.negative(normal)
#     print(normal)
    
    #add normals to relavant vertex indexs 
    vertex_normals[index[0]] += normal
    vertex_normals[index[1]] += normal
    vertex_normals[index[2]] += normal

In [53]:
# print(vertex_normals)

In [54]:
normalized_vector_normals = []
for x in vertex_normals:
    if (np.linalg.norm(x) > 0.0):
        x = x / (np.linalg.norm(x))
    normalized_vector_normals.append(x)

In [55]:
# print(normalized_vector_normals)

In [56]:
# print(normalized_vector_normals[35946])

In [57]:
# print(plydata['vertex'][35878])

In [58]:
result = []
i = 0
while i < count:
    result.append([plydata['vertex'][i], normalized_vector_normals[i]])
    i += 1

In [59]:
# print(result[0][1][0])

In [60]:
f = open("dragon.xyz", "w+")

In [61]:
f.write(str(count) + "\n")

7

In [62]:
for x in result:
    f.write(str(x[0][0]) + " ")
    f.write(str(x[0][1]) + " ")
    f.write(str(x[0][2]) + " ")
    f.write(str(x[1][0]) + " ")
    f.write(str(x[1][1]) + " ")
    f.write(str(x[1][2]) + "\n")

In [63]:
f.close()